In [1]:
# Morphosyntactic marker system (IPI PAN):
# http://nlp.ipipan.waw.pl/CORPUS/znakowanie.pdf
# http://nkjp.pl/poliqarp/help/plse2.html

In [2]:
import numpy as np
import pandas as pd

import morfeusz2
from IPython.display import display, Markdown

In [3]:
def filter_empty_strings(str_list):
    return list(filter(None, str_list))

def index_items(str_list):
    indices = []
    for i in range(len(str_list)):
        indices.append(i)
    return indices

def get_list_item(str_list, item_idx):
    return str_list[item_idx]

In [4]:
data_train_exp = pd.read_csv("../../data/outputs/feature_engineering/train/expected.csv")

In [5]:
base_items_df = data_train_exp.copy()
base_items_df["text_split"] = base_items_df["FixedOutput"].apply(str.split)
base_items_df["text_split"] = base_items_df["text_split"].apply(filter_empty_strings)
base_items_df["item_index"] = base_items_df["text_split"].apply(index_items)
base_items_df = base_items_df.explode("item_index").reset_index(drop=True)
base_items_df["item"] = base_items_df.apply(
    lambda x: get_list_item(x.text_split, x.item_index), axis=1
)

In [6]:
base_items_df

,FileId,FixedOutput,text_split,item_index,item
0,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,"[w, wywiadzie, dla, polski, jarosław, kaczyńsk...",0,w
1,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,"[w, wywiadzie, dla, polski, jarosław, kaczyńsk...",1,wywiadzie
2,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,"[w, wywiadzie, dla, polski, jarosław, kaczyńsk...",2,dla
3,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,"[w, wywiadzie, dla, polski, jarosław, kaczyńsk...",3,polski
4,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,"[w, wywiadzie, dla, polski, jarosław, kaczyńsk...",4,jarosław
...,...,...,...,...,...
184458,wikinews195323,29 lutego na wydziale matematyki i informatyki...,"[29, lutego, na, wydziale, matematyki, i, info...",179,chopina
184459,wikinews195323,29 lutego na wydziale matematyki i informatyki...,"[29, lutego, na, wydziale, matematyki, i, info...",180,30
184460,wikinews195323,29 lutego na wydziale matematyki i informatyki...,"[29, lutego, na, wydziale, matematyki, i, info...",181,wstęp
184461,wikinews195323,29 lutego na wydziale matematyki i informatyki...,"[29, lutego, na, wydziale, matematyki, i, info...",182,wolny


### Adding columns with preceding and following words

In [7]:
prev_items_df = base_items_df.merge(
    base_items_df[["item_index", "item"]]
    .shift(1)
    .rename({"item": "prev_item", "item_index": "prev_item_index"}, axis=1),
    left_index=True,
    right_index=True,
)

items_df = prev_items_df.merge(
    base_items_df[["item_index", "item"]]
    .shift(-1)
    .rename({"item": "next_item", "item_index": "next_item_index"}, axis=1),
    left_index=True,
    right_index=True,
)

prev_cond_mask = items_df["item_index"] == 0
next_cond_mask = items_df["next_item_index"] == 0

items_df.loc[prev_cond_mask, "prev_item"] = np.nan
items_df.loc[next_cond_mask, "next_item"] = np.nan

items_df = items_df.drop(["prev_item_index", "next_item_index", "text_split"], axis=1)

In [8]:
items_df

,FileId,FixedOutput,item_index,item,prev_item,next_item
0,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,0,w,NaN,wywiadzie
1,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,1,wywiadzie,w,dla
2,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,2,dla,wywiadzie,polski
3,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,3,polski,dla,jarosław
4,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,4,jarosław,polski,kaczyński
...,...,...,...,...,...,...
184458,wikinews195323,29 lutego na wydziale matematyki i informatyki...,179,chopina,.,30
184459,wikinews195323,29 lutego na wydziale matematyki i informatyki...,180,30,chopina,wstęp
184460,wikinews195323,29 lutego na wydziale matematyki i informatyki...,181,wstęp,30,wolny
184461,wikinews195323,29 lutego na wydziale matematyki i informatyki...,182,wolny,wstęp,.


### Morphological analysis application

In [9]:
morf = morfeusz2.Morfeusz(analyse=True, generate=False, expand_tags=False)

morf_df = items_df[["item"]].copy()
morf_df["morf_analysis"] = morf_df["item"].apply(morf.analyse)

In [10]:
# extracting relevant information from the output
morf_df = morf_df.explode("morf_analysis").reset_index(drop=True)
morf_df['morf_analysis'] = morf_df['morf_analysis'].str[2]
morf_df["tag"] = morf_df['morf_analysis'].str[2]
morf_df = morf_df.drop('morf_analysis', axis=1)

In [11]:
morf_df

,item,tag
0,w,prep:acc:nwok
1,w,prep:loc:nwok
2,wywiadzie,subst:sg:loc:m3
3,wywiadzie,subst:sg:voc:m3
4,dla,prep:gen
...,...,...
374641,wolny,adj:sg:acc:m3:pos
374642,wolny,adj:sg:nom.voc:m1.m2.m3:pos
374643,wolny,adj:sg:acc:m3:pos
374644,wolny,adj:sg:nom.voc:m1.m2.m3:pos


In [12]:
# checking the words unrecognized by Morpheusz2 (verification whether data cleaning process was correct)
df = morf_df.copy()
ignored_items = df.loc[df["tag"] == "ign"]["item"].value_counts()

for item in ignored_items.index:
    print(item)

ignored_items_df = ignored_items.reset_index().rename(
    {"item": "count", "index": "word"}, axis=1
)
ignored_items_df.to_csv(
    "../../data/outputs/feature_engineering/ignored_items_df.csv", index=False
)

# drop unrecognized items (listed below)
df = df.loc[df["tag"] != "ign"]

r
np
the
godz
of
św
tzw
ks
wikimedia
moze
prof
itp
fukushima
tvn24
itd
n
wikinews
world
commons
f
byc
directx
jesli
obama
displaystyle
skype
times
gadu
sagi2007
good
esa
prosze
sts
tj
ktos
wst
creative
barroso
wikipedystów
mozna
cos
il
nauru
plusfon
cov
live
ćwiąkalski
ubuntu
festival
psp
or
porak
bylo
xx236
play
mark
firefoksa
jpg
server
whatever
marienthal
fernando
etc
lorber
eric
płd
pirx
byla
avm
lgbt
bros
an
stage
trappist
miedwiediew
jeff
phoenix
wiklol
delaya
orange
guardian
pologne
dzong
csc
royal
kastoreum
starowce
gazprom
byl
ency
gg
solidarnosc
barack
gazieta
ditalia
network
chris
newsweek
lng
przeciez
troche
menezesa
borewicz
albertus
onet
maest
sdpl
domo
kashechkin
niz
alonso
life
andrew
eu
waziristanie
dw
mohammad
canal
googlea
abu
wz
wikipedyści
symantec
roo72
oczywiscie
płn
blackfish
ww
alessandro
zwiazku
milingo
kaufland
wikiprojekt
church
netii
pd
petacchi
prow
sun
dlr
daiichi
ns
cantoro
gazpromu
bansp
painkiller
lorenzo
second
inż
mingory
space
electric
slasku
ec725


szkoleniowo
prześladowaniw
motosportu
gazpromeksport
number
virgo
brdzo
villers
h1n1
burchillu
zapasiewicza
humbuckera
trentemller
shutter
gottlieb
hernandeza
dgse
zigzap
ammena
isłanowa
bajm
todd
kzwmet
gazpromowi
artefact
moissan
wyrówania
sasuke
newsweeku
kimiemu
festivalu
ashley
analize
wikipedystapsjulek
ahmadinażeda
dostawiac
boszniacka
kejrouz
zabriskie
teatralno
pewuobsługa
nasdaq
wogóle
miesiączny
mjt
odincovo
madridistas
poprostu
duuuzo
dolaczyc
skrzydlami
joymaster
łoł
bystoel
rozwijac
verfolgung
occurrence
zentrum
btpk
drosophila
telecoms
schetynę
vulcanologia
vigilium
tarbunka
hesja
ffmalan
sophie
wuxi
aol
sereną
pawmakprzydałoby
tąd
istilidion
grandmont
komentaz
ksymil
slyszalna
brancion
lc1
reprezentancja
dragonegg
slavka
zę
stewart
beit
0v
rownież
dictionary
penas
abdullah
piastu
materazzi
najpojemnijeszych
guard
mesajah
advanced
jurgita
ditlenku
evgeni
żomart
evan
partnership
czakwetadze
jboss
damjana
iłowajsku
jasra
ptrtlr
porządkowedefinicja
gomes
rjtskądrze
pouto
fi

uśmierzaczy
mayflower
siltala
neilla
compostela
sat
filmaster
liam
samipałatyńsku
ketlinga
grohmana
frances
lgbtqia
malaheedh
volum
bijuu
openshift
arthus
kautzkich
migatu
home
emmmm
przyeszedł
rodrguez
peochar
ziemianstwo
delitzsch
instead
lomzynskiego
moncych
velupillai
tokajew
skangur
antiqua
użyszkodników
kreske
muhammad
zdjęciamimario58
co2
krabik
perić
klementyńskiej
used
branislav
prmoeniowania
lapacz
warhammer
rozważalem
nortomu
niginie
dave
musharraf
tramwaji
artykulacy
musaba
kenwyne
starscream
emkaffe
kilkudzisięciu
saporta
opendns
googl
mwalimu
fwioo
ledger
abbas
ledesma
wulfstan
0x1ffb23p
environment
lwb
podejsc
spowrotem
kuniszewicz
maelstrom
kashechkina
epitestosteronu
mazovii
dynatek
phipps
protokananejski
hs
dodtkowo
przegladaja
bisauu
qatar
fonobaru
tpsa
games
walks
barratt
seck
colea
rekacji
kirsten
rosajakie
wzbogazenie
ipku
tallis
rownowaznych
ivica
guus
wedycji
jonathan
hack
motif
kangasniemi
sauber
estrack
kyung
nawaf
waclawdzieciol
1017bq
mlodziezowej
niedociąni

In [13]:
# checking the punctuation marks recognized by Morpheusz2
df.loc[df["tag"]=="interp"]["item"].value_counts()

.    10372
,    10069
-     2422
:      897
?      768
!      114
Name: item, dtype: int64

In [14]:
tmp = df['tag'].str.split(':', 1, expand=True).rename(
    {0: 'grammatical_class', 1:'grammatical_categories'},
    axis=1
    )

whitelist=['nom','gen','dat','acc','inst','loc','voc','imperf','perf']
pat = r'\b(?:{})\b'.format('|'.join(whitelist))
tmp['grammatical_categories']= tmp['grammatical_categories'].str.findall('(' + pat + ')').str.join('.')
df = pd.concat([df, tmp], axis=1)

In [15]:
df

,item,tag,grammatical_class,grammatical_categories
0,w,prep:acc:nwok,prep,acc
1,w,prep:loc:nwok,prep,loc
2,wywiadzie,subst:sg:loc:m3,subst,loc
3,wywiadzie,subst:sg:voc:m3,subst,voc
4,dla,prep:gen,prep,gen
...,...,...,...,...
374641,wolny,adj:sg:acc:m3:pos,adj,acc
374642,wolny,adj:sg:nom.voc:m1.m2.m3:pos,adj,nom.voc
374643,wolny,adj:sg:acc:m3:pos,adj,acc
374644,wolny,adj:sg:nom.voc:m1.m2.m3:pos,adj,nom.voc


In [16]:
grammatical_categories_dict = (
    df.drop_duplicates(subset=["item", "grammatical_class"])
    .sort_values(["item", "grammatical_class"], ascending=True)
    .groupby(["item"])["grammatical_class"]
    .apply(";".join)
    .reset_index()
)

In [17]:
grammatical_categories_dict

,item,grammatical_class
0,!,interp
1,",",interp
2,-,interp
3,.,interp
4,0,dig
...,...,...
31268,żądanie,ger;subst
31269,żądaniem,ger;subst
31270,żądać,inf
31271,żądał,praet


### Concatenate enriched frame with morphological analysis results

In [18]:
res_df = items_df.merge(grammatical_categories_dict, on="item", how="left").rename(
    {"grammatical_class": "item_classes"}, axis=1
)
res_df = res_df.merge(
    grammatical_categories_dict, left_on="prev_item", right_on="item", how="left"
).rename({"grammatical_class": "prev_item_classes"}, axis=1)
res_df = res_df.merge(
    grammatical_categories_dict, left_on="next_item", right_on="item", how="left"
).rename({"grammatical_class": "next_item_classes"}, axis=1)
res_df = res_df.drop(["item", "item_y"], axis=1).rename({"item_x": "item"}, axis=1)


### Check the morphological environment of punctuation marks

In [19]:
punctuations_df = res_df.loc[res_df["item_classes"] == "interp"].reset_index()
punctuations_df = punctuations_df[
    [
        "FileId",
        "FixedOutput",
        "item",
        "prev_item",
        "prev_item_classes",
        "next_item",
        "next_item_classes",
    ]
]

punctuations_df

,FileId,FixedOutput,item,prev_item,prev_item_classes,next_item,next_item_classes
0,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,",",podkreślił,praet,że,comp;part
1,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,.,państwa,subst,to,adj;comp;part;pred;subst
2,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,",",wszystko,adj;adv;subst,co,comp;part;prep;subst
3,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,-,powiedzieć,inf,odpowiedział,praet
4,wikinews228460,w wywiadzie dla polski jarosław kaczyński podk...,",",odpowiedział,praet,gdy,adv;comp
...,...,...,...,...,...,...,...
24637,wikinews195323,29 lutego na wydziale matematyki i informatyki...,",",społecznością,subst,administrowaniem,ger
24638,wikinews195323,29 lutego na wydziale matematyki i informatyki...,",",administrowaniem,ger,jak,adv;comp;conj;prep;subst
24639,wikinews195323,29 lutego na wydziale matematyki i informatyki...,.,wikipediami,subst,wykład,subst
24640,wikinews195323,29 lutego na wydziale matematyki i informatyki...,.,ul,subst,chopina,subst


In [20]:
# re-check the target classes balance
punctuations_df["item"].value_counts()

.    10372
,    10069
-     2422
:      897
?      768
!      114
Name: item, dtype: int64

In [21]:
target_classes = [',', '.', '-', ':', '?', '!']

In [22]:
# check the items preceeding individual punctuation marks
res_dict = {}
for punct_symb in target_classes:
    display(Markdown(f'#### Punctuation mark: "{punct_symb}"'))
    df = punctuations_df.loc[punctuations_df["item"] == punct_symb].reset_index()
    display(pd.DataFrame(df["prev_item"].value_counts()).head(20))

#### Punctuation mark: ","

,prev_item
się,91
to,74
tym,59
powiedział,56
roku,54
1,40
osób,38
tego,32
2,31
również,31


#### Punctuation mark: "."

,prev_item
m,151
r,135
in,129
roku,108
ok,90
np,66
tys,63
osób,55
ul,48
godz,46


#### Punctuation mark: "-"

...

,prev_item
wikipedia,22
20,17
3,15
16,14
18,14
zgłasza,14
15,13
13,12
1,12
się,9


#### Punctuation mark: "?"

,prev_item
nie,9
chodzi,7
artykule,6
źródła,5
było,5
co,5
zaciekle,4
błąd,4
prawda,4
dlaczego,4


#### Punctuation mark: "!"

,prev_item
pozdrawiam,5
się,4
konta,4
witam,3
wstyd,2
być,2
dalej,2
błąd,2
przyjaciele,1
tworzył,1


In [23]:
# check the items following individual punctuation marks
res_dict = {}
for punct_symb in target_classes:
    display(Markdown(f'#### Punctuation mark: "{punct_symb}"'))
    df = punctuations_df.loc[punctuations_df["item"] == punct_symb].reset_index()
    display(pd.DataFrame(df["next_item"].value_counts()).head(20))

#### Punctuation mark: ","

,next_item
że,1356
a,575
w,317
ale,245
który,240
które,224
która,160
to,160
co,120
jak,118


#### Punctuation mark: "."

,next_item
w,700
na,203
in,139
nie,120
po,115
to,115
z,113
do,109
według,87
jak,77


#### Punctuation mark: "-"

...

,next_item
00,59
30,20
w,19
zgłoś,19
1,18
2,10
0,9
polska,7
3,7
nie,7


#### Punctuation mark: "?"

,next_item
czy,31
w,28
to,23
nie,22
a,19
bo,12
z,10
przecież,10
na,10
tak,10


#### Punctuation mark: "!"

,next_item
w,6
co,5
.,4
należy,4
",",3
do,3
jak,2
ja,2
to,2
nie,2


In [24]:
# Conclusions:
# 1. there is a need to handle abbreviations within the baseline model
# 2. data cleaning seems to be ok
# 3. certainty: abbreviations for dots, two-part expressions for dashes,